# Generate a mean-field (orbital) and CI representation for the oxygen ground configuration

In [1]:
using JAC

Various *ab-initio* many-electron methods have been developed over the years to describe the level structure and properties of atoms and ions with -- more or less -- simple shell structures. These methods include configurations interaction (CI), many-body perturbation theory (MBPT), coupled-cluster (CC) theory, correlation potential (CP), the multiconfigurational Dirac-Hartree Fock (MCDF) representations and are all typically based on some *self-consistent-field* (SCF).  In JAC, we distinguish between different representations of atomic wave (and Green) functions to facilitate the access to the different methods. For the sake of convinience, we here include also the concept of a self-consistent orbital set as perhaps the most basic *representation* of atomic levels and multiplets.

In this tutorial, we shall show how one can easily generate a mean-field (orbital) representation as well as a CI representation, making use of these orbitals. We shall consider the levels of the $1s^2 2s^2 2p^4$ ground configuration of oxygen as a simple but typical example and shall later extent this concept to perform a CI expansion with (virtual) excitations into the $n=3$ shells.

The computations of all *atomic representations* are controlled by the `struct Atomic.Representation` which is based on a name, nuclear model, (radial) grid as well as set of *reference configurations*. Let us first consider how such a representation is defined inernally:

In [2]:
? Atomic.Representation

`struct  Atomic.Representation`       ... a struct for defining an atomic representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`Atomic.Representation()`  ... constructor for an 'empty' instance of the a variable::Atomic.Representation


The particular choice of an atomic representation is made by the `repType::AbstractRepresentationType` which presently may take the following parameters:

In [3]:
?Atomic.AbstractRepresentationType

`abstract type Atomic.AbstractRepresentationType`      ... defines an abstract type and a number of data types to work with and distinguish different atomic representations; see also:

```
+ struct MeanFieldBasis       ... to represent (and generate) a mean-field basis and, especially, a set of (mean-field) orbitals.
+ struct CiExpansion          ... to represent (and generate) a configuration-interaction representation.
+ struct RasExpansion         ... to represent (and generate) a restricted active-space representation.
+ struct GreenFunction        ... to represent (and generate) an approximate (many-electron) Green functions.
```


Since this is an *abstract type*, we must always specify on of the explicit types together with its parameters as, for instance, a `MeanFieldBasis`, and which is purely determined by its settings:

In [4]:
?MeanFieldBasis

search: MeanFieldBasis



`struct  Atomic.MeanFieldBasis  <:  AbstractRepresentationType`       ... a struct to represent (and generate) a mean-field orbital basis.

```
+ settings         ::Atomic.MeanFieldSettings      ... Settings for the given mean-field orbital basis
```


In [5]:
?Atomic.MeanFieldSettings

`struct  Atomic.MeanFieldSettings`       ... a struct for defining the settings for a mean-field basis (orbital) representation.

```
+ methodScf            ::String             ... Specify the SCF method: ["meanDFS", "meanHS"].
```

---

`Atomic.MeanFieldSettings()`  ... constructor for setting the default values.


With this background, we can now readily compute a mean-field (orbital) basis for the oxygen ground configuration by:

In [6]:
name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
refConfigs  = [Configuration("[He] 2s^2 2p^4")]
mfSettings  = MeanFieldSettings()

methodScf:                meanDFS  


In [7]:
wa          = Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, MeanFieldBasis(mfSettings) )

Atomic representation:   Oxygen 1s^2 2s^2 2p^4 ground configuration for Z = 8.0 and with reference configurations: 
   1s^2 2s^2 2p^4 ,  
representation type:   Mean-field orbital basis for a meanDFS SCF field:
  
nuclearModel:          Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
grid:                  Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = JAC.Radial.MeshGL(), ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]  


In JAC, an atomic representation is always *generate*'d:

In [8]:
wb = generate(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^4 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^3 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.20273042e+01    -3.20273113e+01    +2.20440921e-07    
      2      2s_1/2      -8.00853682e+00    -8.00853549e+00    -1.65615927e-07    
     

Dict{String,Any} with 1 entry:
  "mean-field basis" => Atomic basis:  5 CSF defined for 8 elecrons.…

As seen from the last line of this printout, the outcome::Dict{String,Any} just contains the requested basis and, hence, set of orbital:

In [9]:
orbitals    = wb["mean-field basis"].orbitals

Dict{Subshell,Orbital} with 4 entries:
  2s_1/2 => Bound-state orbital 2s_1/2 with energy -0.8216909939585607 a.u. is …
  2p_1/2 => Bound-state orbital 2p_1/2 with energy -0.28964120578908725 a.u. is…
  2p_3/2 => Bound-state orbital 2p_3/2 with energy -0.2883069592711447 a.u. is …
  1s_1/2 => Bound-state orbital 1s_1/2 with energy -18.705283830406817 a.u. is …

We can apply this *mean-field basis* in order to generate a CI representations for all levels of the $1s^2 2s^2 2p^4$ ground configuration of oxygen; to this end, we first define the corresponding *settings* and (no further) *excitations*,

In [10]:
ciSettings  = CiSettings(true, false, Int64[], false, LevelSymmetry[] )

breitCI:                  true  
selectLevelsCI:           false  
selectedLevelsCI:         Int64[]  
selectSymmetriesCI:       false  
selectedSymmetriesCI:     LevelSymmetry[]  


In [11]:
excitations = RasStep()


CI or RAS step with 0 (explicitly) frozen shell(s): Shell[]  ... and virtual excitations


and before we follow very similar lines as above:

In [12]:
wc          = Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, 
                             CiExpansion(orbitals, excitations, ciSettings) )

Atomic representation:   Oxygen 1s^2 2s^2 2p^4 ground configuration for Z = 8.0 and with reference configurations: 
   1s^2 2s^2 2p^4 ,  
representation type:   CI expansion with (additional) excitations:

CI or RAS step with 0 (explicitly) frozen shell(s): Shell[]  ... and virtual excitations
  
... and the current settings:
breitCI:                  true  
selectLevelsCI:           false  
selectedLevelsCI:         Int64[]  
selectSymmetriesCI:       false  
selectedSymmetriesCI:     LevelSymmetry[]  
  
  
nuclearModel:          Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
grid:                  Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = JAC.Radial.MeshGL(), ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1

In [13]:
wd = generate(wc, output=true)

*** Level symmetries = LevelSymmetry[0 +, 1 +, 2 +] 
>> include Configuration: 1s^2 2s^2 2p^4 
(Re-) Define a new standard subshell list.
Construct a basis with 5 CSF for J^P = LevelSymmetry[0 +, 1 +, 2 +] with 4 subshells: 1s_1/2  2s_1/2 ...  2p_1/2  2p_3/2
Compute CI matrix of dimension 2 x 2 for the symmetry 0^+ ...   ... done.
Compute CI matrix of dimension 1 x 1 for the symmetry 1^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    2 +     -7.484030990301846e+01    -2.036508562627464e+03    -2.036508562627464e+03 
     2    1 +     -7.483958197648447e+01    -2.036488754737553e+03    -2.036488754737553e+03 
     3    0 +     -7.483926015247478e+01    -2.036479997460195e+03    -2.036479997460195e+03 
     4    2 +     -7.476281511712232e+01    -2.034399822093907e+03    -2.034399822093907e+03 
     5    0 +     -7.464699371606233e+01  

Dict{String,Any} with 1 entry:
  "CI multiplet" => name:        CI multiplet:  …

In this case, the outcome::Dict{String,Any} describes the requested (ground configuration) multiplet with the levels:

In [14]:
levels     = wd["CI multiplet"].levels

5-element Array{Level,1}:
 Level: J = 2, M = 2, parity = plus, index = 1 
energy:         -74.84030990301846  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 0.572421, 0.0, 0.81996]  
   
 Level: J = 1, M = 1, parity = plus, index = 2 
energy:         -74.83958197648447  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 1.0, 0.0, 0.0, 0.0]  
            
 Level: J = 0, M = 0, parity = plus, index = 3 
energy:         -74.83926015247478  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [-0.813494, 0.0, 0.0, 0.581573, 0.0]  
 
 Level: J = 2, M = 2, parity = plus, index = 4 
energy:         -74.76281511712232  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 0.81996, 0.0, -0.572421]  
  
 Level: J = 0, M = 0, parity = plus, index = 5 
energy:         -74.64699371606233  
relativeO

# **This nootebook is still under construction !!!**